In [1]:
cd /orange/adamginsburg/salt/sanhueza

/orange/adamginsburg/salt/sanhueza


In [2]:
%matplotlib inline
import pylab as pl
pl.rcParams['figure.facecolor'] = 'w'

In [3]:
from astropy import wcs
from spectral_cube import SpectralCube
import glob
from astropy import units as u
from astropy.io import fits

In [4]:
from dask.diagnostics import ProgressBar
pbar = ProgressBar()

In [5]:
import regions
import numpy as np
import json

In [6]:
import warnings
warnings.filterwarnings('ignore')

In [7]:
non_sanhueza_flist = ['/orange/adamginsburg/salt/G351/G351_spw0_selfcal2.fits',
                      '/orange/adamginsburg/salt/I16547/IRAS16547.Band6.spw0.selfcal.lowcontsub.rp.image.pbcor.fits',
                      '/orange/adamginsburg/salt/maud/G17_SPW0_2017.image.fits',
                      '/orange/adamginsburg/salt/s255ir/S255IR-SMA1_sci.spw3.cube.I.zoom.manual.JvM.image.pbcor.fits']

In [8]:
try:
    with open('fovs_meta.json', 'r') as fh:
        fovs = json.load(fh)
except Exception:
    fovs = {}
try:
    with open('beams_meta.json', 'r') as fh:
        beams = json.load(fh)    
except Exception:
    beams = {}
try:
    with open('spectralresolution.json', 'r') as fh:
        spectralresolution = json.load(fh)    
except Exception:
    spectralresolution = {}

for fn in glob.glob("*/*spw3*small.fits") + non_sanhueza_flist:
    name = os.path.basename(os.path.dirname(fn))
    if name not in beams:
        cube = SpectralCube.read(fn)
        vv = cube.with_spectral_unit(u.km/u.s, velocity_convention='radio').spectral_axis
        dv = np.abs(vv[1]-vv[0])
        pixscale = wcs.utils.proj_plane_pixel_scales(cube.wcs.celestial)*u.deg
        if hasattr(cube, 'beam'):
            print(name, os.path.dirname(fn), (pixscale*cube.shape[1:]).to(u.arcsec), cube.beam, )
            beam = cube.beam
        else:
            print(name, os.path.dirname(fn), (pixscale*cube.shape[1:]).to(u.arcsec),
                  cube.beams.common_beam(max_iter=20, max_epsilon=0.01))
            beam = cube.beams.common_beam(max_iter=20, max_epsilon=0.01)
        beams[name] = list(map(float, (beam.major.to(u.arcsec).value, beam.minor.to(u.arcsec).value, beam.pa.to(u.deg).value)))
        fovs[name] = list(map(float, (pixscale*cube.shape[1:]).to(u.arcsec).value))
        spectralresolution[name] = dv.to(u.km/u.s).value
        with open('fovs_meta.json', 'w') as fh:
            json.dump(fovs, fh)
        with open('beams_meta.json', 'w') as fh:
            json.dump(beams, fh)
        with open('spectralresolution.json', 'w') as fh:
            json.dump(spectralresolution, fh)                

In [9]:
non_sanhueza_full_flist = ([y  for x in ['/orange/adamginsburg/salt/G351/G351_spw[012345]_selfcal2.fits',
                      '/orange/adamginsburg/salt/I16547/IRAS16547.Band6.spw[012345].selfcal.lowcontsub.rp.image.pbcor.fits',
                      '/orange/adamginsburg/salt/maud/G17_SPW[01234]_2017.image.fits',
                      '/orange/adamginsburg/salt/s255ir/S255IR-SMA1_sci.spw[01234].cube.I.zoom.manual.JvM.image.pbcor.fits']
                            for y in glob.glob(x)])
non_sanhueza_full_flist

['/orange/adamginsburg/salt/G351/G351_spw2_selfcal2.fits',
 '/orange/adamginsburg/salt/G351/G351_spw0_selfcal2.fits',
 '/orange/adamginsburg/salt/G351/G351_spw1_selfcal2.fits',
 '/orange/adamginsburg/salt/G351/G351_spw3_selfcal2.fits',
 '/orange/adamginsburg/salt/I16547/IRAS16547.Band6.spw1.selfcal.lowcontsub.rp.image.pbcor.fits',
 '/orange/adamginsburg/salt/I16547/IRAS16547.Band6.spw3.selfcal.lowcontsub.rp.image.pbcor.fits',
 '/orange/adamginsburg/salt/I16547/IRAS16547.Band6.spw2.selfcal.lowcontsub.rp.image.pbcor.fits',
 '/orange/adamginsburg/salt/I16547/IRAS16547.Band6.spw0.selfcal.lowcontsub.rp.image.pbcor.fits',
 '/orange/adamginsburg/salt/maud/G17_SPW2_2017.image.fits',
 '/orange/adamginsburg/salt/maud/G17_SPW0_2017.image.fits',
 '/orange/adamginsburg/salt/maud/G17_SPW3_2017.image.fits',
 '/orange/adamginsburg/salt/maud/G17_SPW1_2017.image.fits',
 '/orange/adamginsburg/salt/s255ir/S255IR-SMA1_sci.spw3.cube.I.zoom.manual.JvM.image.pbcor.fits']

In [10]:
try:
    with open('freqcov.json', 'r') as fh:
        freqcov = json.load(fh)
except Exception:
    freqcov = {}

    
for fn in glob.glob("*/*spw[01234]*small.fits") + non_sanhueza_full_flist:
    name = os.path.basename(os.path.dirname(fn))

    cube = SpectralCube.read(fn).with_spectral_unit(u.GHz)

    if name not in freqcov:
        freqcov[name] = [(cube.spectral_axis.min().value, cube.spectral_axis.max().value)]
    else:
        freqcov[name] += [(cube.spectral_axis.min().value, cube.spectral_axis.max().value)]
    with open('freqcov.json', 'w') as fh:
        json.dump(freqcov, fh)

In [11]:
beams

{'NGC6334I': [0.07804282009601593, 0.051630012691020966, 70.59971618652344],
 'G5.89': [0.05795148657317544, 0.04095939016036121, 66.31839430366688],
 'IRAS16547': [0.07686583218914345, 0.05244795041329405, 126.31673324398756],
 'G333': [0.07612335681915283, 0.058185502886772156, 58.16907501220703],
 'W33A': [0.0966954454779625, 0.06535904854536057, -83.22697448730469],
 'I16562': [0.09567944070181846, 0.05012410463076423, 82.57850646972655],
 'G335': [0.07347185909748077, 0.06121784448623657, 50.99840545654297],
 'G34.43': [0.095871701836586, 0.07116999477148056, 61.10786819458008],
 'IRAS18162': [0.09307271242141724, 0.06486117839813232, -85.60118865966797],
 'G351.77': [0.12185147188038378, 0.06738864666347463, 113.00846099853513],
 'NGC6334IN': [0.07778939604759216, 0.051596663892269135, 70.34847259521484],
 'G29.96': [0.09104029089212418, 0.06827182322740555, 65.14855194091797],
 'G11': [0.09511609974898803, 0.06500983670362831, 96.09446716308595],
 'IRAS18089': [0.058114132738534

In [12]:
mkdir avspectra

mkdir: cannot create directory ‘avspectra’: File exists


In [13]:
import time

In [17]:
try:
    with open('stddev_meta.json', 'r') as fh:
        stddev = json.load(fh)
except Exception as ex:
    print(ex)
    stdddev = {}
try:
    with open('fivesig_meta.json', 'r') as fh:
        nfivesigma = json.load(fh)    
except Exception as ex:
    print(ex)
    nfivesigma = {}

for fn in glob.glob("*/*spw3*small.fits") + non_sanhueza_flist:
    print()
    print(fn, end=None)
    name = os.path.basename(os.path.dirname(fn))
    cube = SpectralCube.read(fn, use_dask=True).with_spectral_unit(u.GHz)
    if name not in stddev:
        with cube.use_dask_scheduler('threads', num_workers=4):  
            #rms = np.median(cube.mad_std(axis=0))
            with pbar:
                rms = cube.std()
        rmsK = rms.to(u.K, beam.jtok_equiv(cube.spectral_axis.mean()))
    else:
        rms, rmsK = stddev[name]
        rms = rms * cube.unit
        rmsK = rmsK * u.K
        
    print(f"RMS = {rms}, RMSK = {rmsK}")

    pixscale = wcs.utils.proj_plane_pixel_scales(cube.wcs.celestial)[0]*u.deg
    if hasattr(cube, 'beam'):
        #print(os.path.dirname(fn), (pixscale*cube.shape[1:]).to(u.arcsec), cube.beam, )
        beam = cube.beam
    else:
        #print(os.path.dirname(fn), (pixscale*cube.shape[1:]).to(u.arcsec),
        #      cube.beams.common_beam(max_iter=20, max_epsilon=0.01))
        beam = cube.beams.common_beam(max_iter=20, max_epsilon=0.01)

    stddev[name] = list(map(float, (rms.value, rmsK.value)))
    
    for regfn in glob.glob(f'{os.path.dirname(fn)}/*.reg'):
        t0=time.time()
        regname = os.path.splitext(os.path.basename(regfn))[0]
        reg = regions.Regions.read(regfn)
        if len(reg) > 1:
            # not a cutout region
            continue
        if name in nfivesigma and regname in nfivesigma[name]:
            # done!
            continue
        print(f"region {regname} t={time.time()-t0}")
        
        
        try:
            with cube.use_dask_scheduler('threads', num_workers=4):
                with pbar:
                    scube = cube.subcube_from_regions(reg)
                    if scube.size >= 1e8:
                        print(f"Skipping b/c of large size.  t={time.time()-t0}")
                        if name in nfivesigma:
                            nfivesigma[name][regname] = (np.nan, np.nan)
                        else:
                            nfivesigma[name] = {regname: (np.nan, np.nan)}
                        continue
            with scube.use_dask_scheduler('threads', num_workers=4):
                with pbar:
                    cont = scube.sigma_clip_spectrally(3, save_to_tmp_dir=True).median(axis=0)
                    scube -= cont
        except ValueError as ex:
            print(f"Exception:{ex} t={time.time()-t0}")
            continue        


        specname = f'{name}_{regname}'
        specfn = f'avspectra/{specname}.fits'
        if os.path.exists(specfn):
            print(f"Loading mean spectrum from disk t={time.time()-t0}")
            spec = fits.getdata(specfn) * cube.unit
        else:
            print(f"Calculating mean spectrum t={time.time()-t0}")
            with cube.use_dask_scheduler('threads', num_workers=4):
                with pbar:
                    spec = scube.mean(axis=(1,2))
            spec.hdu.writeto(specfn, overwrite=True)

        # scale the RMS by 1/sqrt(nbeams)
        assert len(reg) == 1
        regpix = reg[0].to_pixel(scube.wcs.celestial)
        mask = regpix.to_mask()
        npix = mask.data.sum()
        nbeams = (npix*pixscale**2/beam.sr).decompose().value
        print(f"IDing 5-sigma t={time.time()-t0}")
        fivesigma = spec > 5 * rms / nbeams**0.5
        frac = fivesigma.sum() / spec.size
        print(f'specname={specname:20s} ppbeam = {(beam.sr / pixscale**2).decompose():10.5f}   nbeams={nbeams:10.5f} sqrt(nbeams)={nbeams**0.5:10.5f}  fivesig={fivesigma.sum():10.5f}  frac:{frac:6.3f}  median:{np.nanmedian(spec.value):10.2f}  t={time.time()-t0}')
        if name not in nfivesigma:
            nfivesigma[name] = {regname: (frac, nbeams)}
        else:
            nfivesigma[name][regname] = (frac, nbeams)
            
        fig = pl.figure(1, figsize=(10,8))
        fig.clf()
        ax = fig.gca()
        xx = cube.spectral_axis.to(u.GHz).value
        yy = spec.value
        ax.plot(xx, yy)
        yy[~fivesigma] = np.nan
        ax.plot(xx, yy)
        ax.axhline(0, linestyle=':', color='k', alpha=0.5)
        ax.axhline((1*rms/nbeams**0.5).value, linestyle='--', color='k', alpha=0.5)
        ax.axhline((3*rms/nbeams**0.5).value, linestyle='--', color='k', alpha=0.5)
        ax.axhline((5*rms/nbeams**0.5).value, linestyle='--', color='k', alpha=0.5)
        ax.set_xlabel("Frequency [GHz]")
        ax.set_ylabel(f"Flux Density [{spec.unit.to_string('latex')}]")
        fig.savefig(specfn.replace(".fits", ".png"), bbox_inches='tight')
        print(f"Saved figure t={time.time()-t0}")
                   

    with open('stddev_meta.json', 'w') as fh:
        json.dump(stddev, fh)
    with open('fivesig_meta.json', 'w') as fh:
        json.dump(nfivesigma, fh)    



NGC6334I/NGC6334I.config8.spw3.robust0.5.image_small.fits
RMS = 0.002369687194004655 Jy / beam, RMSK = 19.56815583356257 K

G5.89/G5.89-0.37.config8.spw3.robust0.5.image_small.fits
RMS = 0.0020644988398998976 Jy / beam, RMSK = 19.15587080013167 K

G5.89/G5.89-0.37.config8.spw3.robust0.5.image_cb_small.fits
RMS = 0.0020644988398998976 Jy / beam, RMSK = 19.15587080013167 K

IRAS16547/IRAS165474247_G343.12-0.06.config8.spw3.robust0.5.image_small.fits
RMS = 0.0019507521064952016 Jy / beam, RMSK = 9.283274325274972 K

G333/G333.23-0.06.config8.spw3.robust0.5.image_small.fits
RMS = 0.002056265017017722 Jy / beam, RMSK = 18.410512124895526 K

W33A/W33A.config8.spw3.robust0.5.image_small.fits
RMS = 0.0017960338154807687 Jy / beam, RMSK = 6.548294946945052 K

I16562/IRAS_165623959.config8.spw3.robust0.5.image_small.fits
RMS = 0.002353072864934802 Jy / beam, RMSK = 10.510717169051501 K

G335/G335.579-0.272.config8.spw3.robust0.5.image_small.fits
RMS = 0.0021833227947354317 Jy / beam, RMSK = 20.

In [18]:
stddev

{'NGC6334I': [0.002369687194004655, 19.56815583356257],
 'G5.89': [0.0020644988398998976, 19.15587080013167],
 'IRAS16547': [0.0019507521064952016, 9.283274325274972],
 'G333': [0.002056265017017722, 18.410512124895526],
 'W33A': [0.0017960338154807687, 6.548294946945052],
 'I16562': [0.002353072864934802, 10.510717169051501],
 'G335': [0.0021833227947354317, 20.378854177206982],
 'G34.43': [0.002639893675222993, 9.205689863612143],
 'IRAS18162': [0.0017806871328502893, 6.867361474269863],
 'G351.77': [0.0037437290884554386, 20.240576180582018],
 'NGC6334IN': [0.0021419536788016558, 17.452679199811215],
 'G29.96': [0.0026674962136894464, 9.576853564203217],
 'G11': [0.0017597611295059323, 6.632903397464641],
 'IRAS18089': [0.002085670828819275, 18.44416797957257],
 '/orange/adamginsburg/salt/G351': [0.0006828127661719918, 6.0182504925676525],
 '/orange/adamginsburg/salt/I16547': [0.0006063765613362193,
  23.215141225401332],
 '/orange/adamginsburg/salt/maud': [0.0010918023763224483, 9.

In [19]:
nfivesigma

{'NGC6334I': {'verysmall': [0.07421875, 7.714804169292351],
  'mm1d': [0.10182291666666667, 20.292272784684126],
  'mm1b': [0.23203125, 27.492756676023657],
  'mm2a': [0.23671875, 15.663390283108717],
  'smallcutout': [0.109375, 17.393376672586395],
  'cutout': [nan, nan],
  'mm1_all': [nan, nan],
  'mm2b': [0.07421875, 7.714804169292351]},
 'G5.89': {'diskcandidate2': [0.06484375, 326.2572771797292],
  'diskcandidate1': [0.05130208333333333, 241.70929292450515],
  'edgeondisk': [0.0010416666666666667, 85.58578076315163]},
 'IRAS16547': {'cutout': [0.27890625, 84.09537351955697]},
 'G333': {'cutout3': [0.0, 14.457671592705545],
  'cutout': [0.015364583333333333, 70.5985002448998],
  'cutout2': [0.08463541666666667, 45.15675348111277]},
 'W33A': {'small': [0.16632124352331606, 51.87987872640841],
  'big': [nan, nan]},
 'I16562': {'bigbox': [0.03463541666666667, 284.53724571032313],
  'box': [0.04739583333333333, 81.68115866857677],
  'rrldiskbox': [0.03697916666666667, 25.20367692002995

# Next cell
the goal of the next cell is to go into the averaged spectra (both stacked and not) and compute statistics.


In [20]:
import pyspeckit
from astropy.stats import mad_std

In [21]:
spw_stats = {}
for field in nfivesigma:
    spw_stats[field] = {}
    for disk in nfivesigma[field]:
        spw_stats[field][disk] = {}
        for spw in (0,1,2,3):
            stackfns = (glob.glob(f"{field}/spectra/*{disk}*stack.fits"))
            meanfns = (glob.glob(f"{field}/spectra/*{disk}*meanspec.fits"))
            if len(stackfns) > 0 and len(meanfns) > 0:
                spst = pyspeckit.Spectrum(stackfns[0])
                stackstd = mad_std(spst.data)
                mnst = pyspeckit.Spectrum(meanfns[0])
                meanstd = mad_std(mnst.data)
                spw_stats[field][disk][spw] = {'stack': stackstd,
                                              'mean': meanstd}
                if len(stackfns) > 1 or len(meanfns) > 1:
                    print(stackfns, meanfns)
            else:
                print(f"Skipped {field} {disk} {spw}")
            


Skipped NGC6334I verysmall 0
Skipped NGC6334I verysmall 1
Skipped NGC6334I verysmall 2
Skipped NGC6334I verysmall 3
['NGC6334I/spectra/NGC6334I.config8.spw0.robust0.5.image_cb_smaller_mm1_mm1d_stack.fits', 'NGC6334I/spectra/NGC6334I.config8.spw2.robust0.5.image_cb_smaller_mm1_mm1d_stack.fits', 'NGC6334I/spectra/NGC6334I.config8.spw1.robust0.5.image_cb_smaller_mm1_mm1d_stack.fits', 'NGC6334I/spectra/NGC6334I.config8.spw2.robust0.5.image_cb_smaller_mm1_NGC6334Imm1d_stack.fits', 'NGC6334I/spectra/NGC6334I.config8.spw0.robust0.5.image_cb_smaller_mm1_NGC6334Imm1d_stack.fits', 'NGC6334I/spectra/NGC6334I.config8.spw3.robust0.5.image_cb_smaller_mm1_NGC6334Imm1d_stack.fits', 'NGC6334I/spectra/NGC6334I.config8.spw1.robust0.5.image_cb_smaller_mm1_NGC6334Imm1d_stack.fits', 'NGC6334I/spectra/NGC6334I.config8.spw3.robust0.5.image_cb_smaller_mm1_mm1d_stack.fits'] ['NGC6334I/spectra/NGC6334I.config8.spw2.robust0.5.image_cb_smaller_mm1_NGC6334Imm1d_meanspec.fits', 'NGC6334I/spectra/NGC6334I.config8.spw

In [22]:
spw_stats

{'NGC6334I': {'verysmall': {},
  'mm1d': {0: {'stack': 16.546554357979392, 'mean': 11.026737233194842},
   1: {'stack': 16.546554357979392, 'mean': 11.026737233194842},
   2: {'stack': 16.546554357979392, 'mean': 11.026737233194842},
   3: {'stack': 16.546554357979392, 'mean': 11.026737233194842}},
  'mm1b': {0: {'stack': 12.648056814757078, 'mean': 9.463133421368957},
   1: {'stack': 12.648056814757078, 'mean': 9.463133421368957},
   2: {'stack': 12.648056814757078, 'mean': 9.463133421368957},
   3: {'stack': 12.648056814757078, 'mean': 9.463133421368957}},
  'mm2a': {},
  'smallcutout': {},
  'cutout': {},
  'mm1_all': {},
  'mm2b': {0: {'stack': 8.099679638532221, 'mean': 9.330600332798845},
   1: {'stack': 8.099679638532221, 'mean': 9.330600332798845},
   2: {'stack': 8.099679638532221, 'mean': 9.330600332798845},
   3: {'stack': 8.099679638532221, 'mean': 9.330600332798845}}},
 'G5.89': {'diskcandidate2': {}, 'diskcandidate1': {}, 'edgeondisk': {}},
 'IRAS16547': {'cutout': {}},
 

In [23]:
nfivesigma.keys()

dict_keys(['NGC6334I', 'G5.89', 'IRAS16547', 'G333', 'W33A', 'I16562', 'G335', 'G34.43', 'IRAS18162', 'G351.77', 'NGC6334IN', 'G29.96', 'G11', 'IRAS18089', 'G351', 'I16547', 's255ir'])

In [24]:
beams

{'NGC6334I': [0.07804282009601593, 0.051630012691020966, 70.59971618652344],
 'G5.89': [0.05795148657317544, 0.04095939016036121, 66.31839430366688],
 'IRAS16547': [0.07686583218914345, 0.05244795041329405, 126.31673324398756],
 'G333': [0.07612335681915283, 0.058185502886772156, 58.16907501220703],
 'W33A': [0.0966954454779625, 0.06535904854536057, -83.22697448730469],
 'I16562': [0.09567944070181846, 0.05012410463076423, 82.57850646972655],
 'G335': [0.07347185909748077, 0.06121784448623657, 50.99840545654297],
 'G34.43': [0.095871701836586, 0.07116999477148056, 61.10786819458008],
 'IRAS18162': [0.09307271242141724, 0.06486117839813232, -85.60118865966797],
 'G351.77': [0.12185147188038378, 0.06738864666347463, 113.00846099853513],
 'NGC6334IN': [0.07778939604759216, 0.051596663892269135, 70.34847259521484],
 'G29.96': [0.09104029089212418, 0.06827182322740555, 65.14855194091797],
 'G11': [0.09511609974898803, 0.06500983670362831, 96.09446716308595],
 'IRAS18089': [0.058114132738534

In [25]:
fovs

{'NGC6334I': [10.377, 10.377],
 'G5.89': [7.1800000000005735, 7.1800000000005735],
 'IRAS16547': [5.13000000000041, 5.13000000000041],
 'G333': [6.660000000000533, 6.660000000000533],
 'W33A': [5.13000000000041, 5.13000000000041],
 'I16562': [13.347000000000001, 13.347000000000001],
 'G335': [5.13000000000041, 5.13000000000041],
 'G34.43': [5.329999999999836, 5.329999999999836],
 'IRAS18162': [8.210000000000656, 8.210000000000656],
 'G351.77': [8.527999999999738, 8.527999999999738],
 'NGC6334IN': [15.561, 15.561],
 'G29.96': [5.329999999999836, 5.329999999999836],
 'G11': [5.13000000000041, 5.13000000000041],
 'IRAS18089': [5.13000000000041, 5.13000000000041],
 '/orange/adamginsburg/salt/G351': [12.800000000001024, 12.800000000001024],
 '/orange/adamginsburg/salt/I16547': [8.191999999999181, 8.191999999999181],
 '/orange/adamginsburg/salt/maud': [2.560000000000205, 2.560000000000205],
 '/orange/adamginsburg/salt/s255ir': [2.09999999999988, 2.10000000000006],
 'G351': [12.80000000000102

In [26]:
from astropy.table import Table

In [27]:
tabledata = {key:[] for key in ('field', 'disk', 'bmaj', 'bmin', 'bpa', 'rmsK', 'fivesigfrac', 'nbeams_avgd')}
for field, cutouts in nfivesigma.items():
    for candidate in cutouts:
        tabledata['field'].append( field )
        tabledata['disk'].append( candidate )
        tabledata['bmaj'].append( beams[field][0] )
        tabledata['bmin'].append( beams[field][1] )
        tabledata['bpa'].append( beams[field][2] )
        tabledata['rmsK'].append( stddev[field][1] )
        tabledata['fivesigfrac'].append( cutouts[candidate][0] )
        tabledata['nbeams_avgd'].append( cutouts[candidate][1] )
tbl = Table(tabledata)

In [28]:
tbl

field,disk,bmaj,bmin,bpa,rmsK,fivesigfrac,nbeams_avgd
str9,str16,float64,float64,float64,float64,float64,float64
NGC6334I,verysmall,0.07804282009601593,0.051630012691020966,70.59971618652344,19.56815583356257,0.07421875,7.714804169292351
NGC6334I,mm1d,0.07804282009601593,0.051630012691020966,70.59971618652344,19.56815583356257,0.10182291666666667,20.292272784684126
NGC6334I,mm1b,0.07804282009601593,0.051630012691020966,70.59971618652344,19.56815583356257,0.23203125,27.492756676023657
NGC6334I,mm2a,0.07804282009601593,0.051630012691020966,70.59971618652344,19.56815583356257,0.23671875,15.663390283108717
NGC6334I,smallcutout,0.07804282009601593,0.051630012691020966,70.59971618652344,19.56815583356257,0.109375,17.393376672586395
NGC6334I,cutout,0.07804282009601593,0.051630012691020966,70.59971618652344,19.56815583356257,nan,nan
NGC6334I,mm1_all,0.07804282009601593,0.051630012691020966,70.59971618652344,19.56815583356257,nan,nan
NGC6334I,mm2b,0.07804282009601593,0.051630012691020966,70.59971618652344,19.56815583356257,0.07421875,7.714804169292351
G5.89,diskcandidate2,0.05795148657317544,0.04095939016036121,66.31839430366688,19.15587080013167,0.06484375,326.2572771797292


In [29]:
for row in tbl:
    print(f"\"{row['field']}_{row['disk']}\":")

"NGC6334I_verysmall":
"NGC6334I_mm1d":
"NGC6334I_mm1b":
"NGC6334I_mm2a":
"NGC6334I_smallcutout":
"NGC6334I_cutout":
"NGC6334I_mm1_all":
"NGC6334I_mm2b":
"G5.89_diskcandidate2":
"G5.89_diskcandidate1":
"G5.89_edgeondisk":
"IRAS16547_cutout":
"G333_cutout3":
"G333_cutout":
"G333_cutout2":
"W33A_small":
"W33A_big":
"I16562_bigbox":
"I16562_box":
"I16562_rrldiskbox":
"G335_cutout2":
"G335_cutout":
"G34.43_cutout":
"IRAS18162_cutout2":
"IRAS18162_cutout":
"IRAS18162_GGD27":
"IRAS18162_GGD27_cutout":
"G351.77_cutout3":
"G351.77_disk1":
"G351.77_disk2":
"G351.77_cutout":
"G351.77_cutout2":
"G351.77_disk3":
"NGC6334IN_IN_1":
"NGC6334IN_IN_3_sm":
"NGC6334IN_IN_3":
"G29.96_cutout":
"G11_sourceAsmall":
"G11_sourceAdisk":
"G11_sourceAsmaller":
"G11_sourceA":
"IRAS18089_diskbox":
"IRAS18089_innerdiskbox":
"IRAS18089_continuumcutout":
"IRAS18089_continuumpolygon":
"G351_disk1":
"G351_disk2":
"G351_disk3":
"I16547_I16547A":
"I16547_I16547B":
"s255ir_rrlcenter":
"s255ir_zoom":


In [30]:
disk_name_mapping = {
"NGC6334I_verysmall": "mm2b",
"NGC6334I_mm2b": "mm2b",
"NGC6334I_mm1d": "mm1d",
"NGC6334I_mm1b": "mm1b",
"NGC6334I_mm2a": None, # decided to leave this out "mm2a",
"NGC6334I_smallcutout": None,
"G5.89_diskcandidate2": None,
"G5.89_diskcandidate1": None,
"G5.89_edgeondisk": 'mm15',
"IRAS16547_cutout": None, # we use Tanaka data for this
"G333_cutout3": None,
"G333_cutout": "mm1",
"G333_cutout2": "mm2",
"W33A_small": "mm1-main",
"I16562_bigbox": None,
"I16562_box": None,
"I16562_rrldiskbox": "G345.4938+01.4677",
"G335_cutout2": "ALMA1",
"G34.43_cutout": "mm1",
"IRAS18162_cutout2": "GGD27",
"G351.77_cutout3":None,
"G351.77_disk1": None, # use Beuther "mm1",
"G351.77_disk2": None, # use Beuther "mm2",
"G351.77_disk3": None, # use Beuther "mm12",
'NGC6334IN_IN_1': "SMA1b/d",
'NGC6334IN_IN_3_sm': "SMA6",    
'NGC6334IN_IN_3': None,
'G29.96_cutout': "submm1",
'G11_sourceAsmall': "mm1",
"G11_sourceAdisk": None,
"G11_sourceAsmaller": None,
"G11_sourceA": None,
"IRAS18089_diskbox": None,
"IRAS18089_innerdiskbox": "I18089-1732",
"G351_disk1": "mm1", # these are the Beuther data
"G351_disk2": "mm2",
"G351_disk3": "mm12",
"I16547_I16547A": "A",
"I16547_I16547B": "B",
"s255ir_rrlcenter": "SMA1",
"NGC6334I_cutout": None,
"NGC6334I_mm1_all": None,
"W33A_big": None,
"G335_cutout": None,
"IRAS18162_cutout": None,
"IRAS18162_GGD27": None,
"IRAS18162_GGD27_cutout": None,
"G351.77_cutout": None,
"G351.77_cutout2": None,
"IRAS18089_continuumcutout": None,
"IRAS18089_continuumpolygon": None,
"s255ir_zoom": None,    
}

In [31]:
# This should be blank!!
for row in tbl:
    if f"{row['field']}_{row['disk']}" not in disk_name_mapping:
        print(f"\"{row['field']}_{row['disk']}\": None,")

In [32]:
%run /orange/adamginsburg/salt/Orion_ALMA_2016.1.00165.S/analysis/latex_info.py

In [33]:
latexdict = latexdict.copy()
latexdict['preamble'] = r'''
\centering
\caption{Observation Summary}
\label{tab:obsstats}
'''
latexdict['tablefoot'] = (r'''
\par
Observation properties.  The `Field' name indicates the region of the ALMA pointing.  The `Source Name' is the identifier of the disk candidate examined.
$\theta$ gives the beam parameters.
$\sigma$ is the average noise level of the field, which is averaged down by $N_{beams}^{1/2}$ to give $\sigma_{\rm avg}$, the noise level in the stacked spectrum.
$f(>5\sigma)$ is the fraction of the stacked spectrum that is above five times $\sigma_{avg}$; it is used as a diagnostic of the line crowding in the spectrum covering 219.2--220.8 GHz, which is high for complex-molecule-rich regions.
''')

textbl=tbl.copy()
textbl.rename_column("fivesigfrac", r"$f(>5\sigma)$")
textbl.rename_column("nbeams_avgd", r"$N_{\rm beams}$")
textbl.rename_column("rmsK", r"$\sigma$")
textbl[r'$\sigma$'].unit = u.K
textbl.rename_column("bmin", r"$\theta_{\rm min}$")
textbl.rename_column("bmaj", r"$\theta_{\rm maj}$")
textbl[r"$\theta_{\rm min}$"].unit = u.arcsec
textbl[r"$\theta_{\rm maj}$"].unit = u.arcsec
textbl.rename_column("bpa", "PA")
textbl["PA"].unit = u.deg
textbl.rename_column("field", "Field")
textbl.rename_column("disk", "Source Name")
textbl[r'$\sigma_{\rm avg}$'] = tbl['rmsK'] / tbl['nbeams_avgd']**0.5
textbl[r'$\sigma_{\rm avg}$'].unit=u.K

selrow = [ii for ii, row in enumerate(tbl) if disk_name_mapping[f"{row['field']}_{row['disk']}"] is not None]
textbl['Source Name'] = [disk_name_mapping[f"{row['field']}_{row['disk']}"] for row in tbl]
textbl = textbl[selrow]

formats = {"$f(>5\sigma)$": lambda x: "{0:0.2f}".format(x),
           r"$N_{\rm beams}$": lambda x: f"{int(x):0d}",
           r"$\theta_{\rm min}$": lambda x:f"{x:0.3f}",
           r"$\theta_{\rm maj}$": lambda x:f"{x:0.3f}",
           "PA": lambda x:f"{x:0.1f}",
           r'$\sigma_{\rm avg}$': lambda x:f"{x:0.1f}", 
           '$\sigma$': lambda x:f"{x:0.1f}",
          }


textbl.write('data_summary.tex', format='ascii.latex', formats=formats, latexdict=latexdict, overwrite=True)
textbl

Field,Source Name,$\theta_{\rm maj}$,$\theta_{\rm min}$,PA,$\sigma$,$f(>5\sigma)$,$N_{\rm beams}$,$\sigma_{\rm avg}$
,,arcsec,arcsec,deg,K,,,K
str9,object,float64,float64,float64,float64,float64,float64,float64
NGC6334I,mm2b,0.07804282009601593,0.051630012691020966,70.59971618652344,19.56815583356257,0.07421875,7.714804169292351,7.045104591309969
NGC6334I,mm1d,0.07804282009601593,0.051630012691020966,70.59971618652344,19.56815583356257,0.10182291666666667,20.292272784684126,4.34394734532633
NGC6334I,mm1b,0.07804282009601593,0.051630012691020966,70.59971618652344,19.56815583356257,0.23203125,27.492756676023657,3.7319924293931215
NGC6334I,mm2b,0.07804282009601593,0.051630012691020966,70.59971618652344,19.56815583356257,0.07421875,7.714804169292351,7.045104591309969
G5.89,mm15,0.05795148657317544,0.04095939016036121,66.31839430366688,19.15587080013167,0.0010416666666666667,85.58578076315163,2.070623271825289
G333,mm1,0.07612335681915283,0.058185502886772156,58.16907501220703,18.410512124895526,0.015364583333333333,70.5985002448998,2.191129933131067
G333,mm2,0.07612335681915283,0.058185502886772156,58.16907501220703,18.410512124895526,0.08463541666666667,45.15675348111277,2.7397094890777196
W33A,mm1-main,0.0966954454779625,0.06535904854536057,-83.22697448730469,6.548294946945052,0.16632124352331606,51.87987872640841,0.909135793096374


In [34]:
textbl.colnames

['Field',
 'Source Name',
 '$\\theta_{\\rm maj}$',
 '$\\theta_{\\rm min}$',
 'PA',
 '$\\sigma$',
 '$f(>5\\sigma)$',
 '$N_{\\rm beams}$',
 '$\\sigma_{\\rm avg}$']

In [35]:
pwd

'/orange/adamginsburg/salt/sanhueza'

In [36]:
cat data_summary.tex

\begin{table*}[htp]

\centering
\caption{Observation Summary}
\label{tab:obsstats}

\begin{tabular}{ccccccccc}
\hline \hline
Field & Source Name & $\theta_{\rm maj}$ & $\theta_{\rm min}$ & PA & $\sigma$ & $f(>5\sigma)$ & $N_{\rm beams}$ & $\sigma_{\rm avg}$ \\
 &  & $\mathrm{{}^{\prime\prime}}$ & $\mathrm{{}^{\prime\prime}}$ & $\mathrm{{}^{\circ}}$ & $\mathrm{K}$ &  &  & $\mathrm{K}$ \\
\hline
NGC6334I & mm2b & 0.078 & 0.052 & 70.6 & 19.6 & 0.07 & 7 & 7.0 \\
NGC6334I & mm1d & 0.078 & 0.052 & 70.6 & 19.6 & 0.10 & 20 & 4.3 \\
NGC6334I & mm1b & 0.078 & 0.052 & 70.6 & 19.6 & 0.23 & 27 & 3.7 \\
NGC6334I & mm2b & 0.078 & 0.052 & 70.6 & 19.6 & 0.07 & 7 & 7.0 \\
G5.89 & mm15 & 0.058 & 0.041 & 66.3 & 19.2 & 0.00 & 85 & 2.1 \\
G333 & mm1 & 0.076 & 0.058 & 58.2 & 18.4 & 0.02 & 70 & 2.2 \\
G333 & mm2 & 0.076 & 0.058 & 58.2 & 18.4 & 0.08 & 45 & 2.7 \\
W33A & mm1-main & 0.097 & 0.065 & -83.2 & 6.5 & 0.17 & 51 & 0.9 \\
I16562 & G345.4938+01.4677 & 0.096 & 0.050 & 82.6 & 10.5 & 0.04 & 25 & 2.1 \\
G335

In [37]:
tabledata = {key:[] for key in ('field', 'disk', 'bmaj', 'bmin', 'bpa', 'rmsK', 'fivesigfrac', 'nbeams_avgd')}
for field, cutouts in nfivesigma.items():
    for candidate in cutouts:
        tabledata['field'].append( field )
        tabledata['disk'].append( candidate )
        tabledata['bmaj'].append( beams[field][0] )
        tabledata['bmin'].append( beams[field][1] )
        tabledata['bpa'].append( beams[field][2] )
        tabledata['rmsK'].append( stddev[field][1] )
        tabledata['fivesigfrac'].append( cutouts[candidate][0] )
        tabledata['nbeams_avgd'].append( cutouts[candidate][1] )
tbl = Table(tabledata)